# नामित घटक ओळख (NER)

हा नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) मधून घेतलेला आहे.

या उदाहरणात, आपण [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) या Kaggle वरील डेटासेटचा वापर करून NER मॉडेल कसे प्रशिक्षण द्यायचे ते शिकणार आहोत. पुढे जाण्यापूर्वी, कृपया [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) फाइल सध्याच्या डिरेक्टरीमध्ये डाउनलोड करा.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## डेटासेट तयार करणे

आम्ही डेटासेट वाचून ते एका डेटा फ्रेममध्ये आणण्यापासून सुरुवात करू. तुम्हाला Pandas वापरण्याबद्दल अधिक जाणून घ्यायचे असल्यास, आमच्या [Data Science for Beginners](http://aka.ms/datascience-beginners) मधील [डेटा प्रक्रिया शिकवणी](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) ला भेट द्या.


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

आता आपल्याला शब्दसंग्रहासह तेच करणे आवश्यक आहे. सोप्या पद्धतीसाठी, आपण शब्दांची वारंवारता विचारात न घेता शब्दसंग्रह तयार करू; प्रत्यक्ष जीवनात, आपण Keras व्हेक्टरायझर वापरू शकता आणि शब्दांची संख्या मर्यादित करू शकता.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

आम्हाला प्रशिक्षणासाठी वाक्यांचा डेटासेट तयार करायचा आहे. चला मूळ डेटासेटमधून फेरफटका मारू आणि सर्व स्वतंत्र वाक्ये `X` (शब्दांच्या यादी) आणि `Y` (टोकनची यादी) मध्ये विभाजित करू:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

साधेपणासाठी, आपण सर्व वाक्ये कमाल लांबीपर्यंत 0 टोकनसह भरून काढू. वास्तविक जीवनात, आपण अधिक हुशार रणनीती वापरू शकतो आणि फक्त एका मिनीबॅचमध्ये अनुक्रम भरून काढू.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## टोकन वर्गीकरण नेटवर्क परिभाषित करणे

टोकन वर्गीकरणासाठी आपण दोन-स्तरीय द्विदिशात्मक LSTM नेटवर्क वापरणार आहोत. शेवटच्या LSTM स्तराच्या प्रत्येक आउटपुटवर घन वर्गीकरणकर्ता लागू करण्यासाठी, आपण `TimeDistributed` संरचना वापरणार आहोत, जी LSTM च्या प्रत्येक टप्प्यावर समान घन स्तराची पुनरावृत्ती करते:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

येथे आपण आपल्या डेटासेटसाठी `maxlen` स्पष्टपणे निर्दिष्ट करत आहोत - जर आपल्याला नेटवर्कला बदलत्या लांबीच्या अनुक्रमांचे व्यवस्थापन करण्यास सक्षम करायचे असेल, तर नेटवर्क परिभाषित करताना आपल्याला थोडे अधिक हुशार असणे आवश्यक आहे.

आता आपण मॉडेल प्रशिक्षण देऊया. गतीसाठी, आपण फक्त एकच epoch साठी प्रशिक्षण देणार आहोत, परंतु आपण अधिक वेळासाठी प्रशिक्षण देण्याचा प्रयत्न करू शकता. तसेच, आपण डेटासेटचा काही भाग प्रशिक्षण डेटासेट म्हणून वेगळा ठेवू शकता, जेणेकरून सत्यापन अचूकता निरीक्षण करता येईल.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## निकालाची चाचणी करणे

आता पाहूया की आपला एंटिटी ओळखण्याचा मॉडेल एका नमुना वाक्यावर कसा कार्य करतो:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## मुख्य मुद्दा

साधा LSTM मॉडेल देखील NER मध्ये समाधानकारक परिणाम दाखवतो. मात्र, अधिक चांगले परिणाम मिळवण्यासाठी, तुम्ही BERT सारखे मोठे पूर्व-प्रशिक्षित भाषा मॉडेल्स वापरण्याचा विचार करू शकता. Huggingface Transformers लायब्ररीचा वापर करून NER साठी BERT प्रशिक्षण कसे द्यावे, याचे वर्णन [येथे](https://huggingface.co/course/chapter7/2?fw=pt) केले आहे.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
